# Question 2
### Working with ADQL

In [2]:
# Basic imports
import numpy as np
import matplotlib.pyplot as plt

from astropy.table import Table

In [4]:
# Download the GAIA Data
from astroquery.gaia import Gaia

From the Gaia/2MASS Tables, we need:
- Gaia BP-RP
- Gaia absolute g
- 2Mass J-Ks
- 2Mass Apparent H

In [101]:
# Select relevant columns and join with the 2Mass table. Constrain ra/dec to
# a 1 degree circle from Ruprecht 147
query = """
SELECT gaia.source_id, gaia.ra, gaia.dec, gaia.parallax, gaia.phot_g_mean_mag, 
gaia.bp_rp, tmass.j_m, tmass.ks_m, tmass.h_m
FROM gaiadr3.gaia_source AS gaia
JOIN gaiadr3.tmass_psc_xsc_best_neighbour AS xmatch USING (source_id)
JOIN gaiadr3.tmass_psc_xsc_join AS xjoin USING (clean_tmass_psc_xsc_oid)
JOIN gaiadr1.tmass_original_valid AS tmass ON
   xjoin.original_psc_source_id = tmass.designation
WHERE phot_g_mean_mag < 14
   AND CONTAINS(
     POINT('ICRS', gaia.ra, gaia.dec),
     CIRCLE('ICRS', 289.074, -16.323, 1.0)
   ) = 1;
"""
job = Gaia.launch_job_async(query)
result = job.get_results()
df = result.to_pandas()
display(df.head())
print("Dataframe has shape", df.shape)


INFO: Query finished. [astroquery.utils.tap.core]


,source_id,ra,dec,parallax,phot_g_mean_mag,bp_rp,j_m,ks_m,h_m
0,4088892600474423808,288.225913,-15.744217,0.624552,11.983528,1.559996,10.017,9.218,9.410
1,4088893154546972672,288.271283,-15.709582,0.454480,12.048648,1.703223,9.941,9.005,9.188
2,4088893188906716032,288.254396,-15.713978,0.211160,12.808913,1.847858,10.526,9.503,9.733
3,4087815667506086656,288.512276,-16.874361,0.092825,12.800300,1.850592,10.501,9.492,9.710
4,4087815697559746304,288.520970,-16.874898,0.366892,11.617373,3.226758,8.277,7.032,7.314


Dataframe has shape (3637, 9)


In [102]:
# Manipulate column names to be easier to work with, and add absolute g magnitude
df.columns = ["ID", "RA", "DEC", "parallax", "g_m", "bp_rp", "j_m", "ks_m", "h_m"]
df["g_M"] = df["g_m"]+5*np.log10(df["parallax"]/1000)+5

c:\Users\taase\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
